In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score
import random
import statistics
import skimage.io
import skimage.color


In [ ]:
def transform(x):
  x=(x-np.mean(x,axis=0))/np.std(x,axis=0)
  #print(x.shape)
  return x

In [ ]:
# X, y = make_classification(n_samples=50000, n_features=20, n_informative=10, n_redundant=5,
#                            n_classes=2, weights=[0.7], class_sep=0.6, random_state=15)
b = np.load('/content/drive/MyDrive/bloodmnist.npz')
train_images=b['train_images']
val_images=b['val_images']
test_images=b['test_images']
Y_train=b['train_labels']
Y_test=b['val_labels']
Y_val=b['test_labels']
train_image=[]
test_image=[]
val_image=[]
for i in range(len(train_images)):
  train_image.append(skimage.color.rgb2gray(train_images[i]))
for i in range(len(test_images)):
  test_image.append(skimage.color.rgb2gray(test_images[i]))
for i in range(len(val_images)):
  val_image.append(skimage.color.rgb2gray(val_images[i]))
train_image=np.array(train_image)
test_image=np.array(test_image)
val_image=np.array(val_image)
print(train_image.shape) 
X_train= train_image.reshape((train_images.shape[0],784))
X_val= test_image.reshape((test_images.shape[0],784))
X_test= val_image.reshape((val_images.shape[0],784))
X_train=transform(X_train)
X_val=transform(X_val)
X_test=transform(X_test)
# train_data=np.hstack((train_images,X_train))
# test_data=np.hstack((test_images,X_test))
# val_data=np.hstack((val_images,X_val))
X_train = np.hstack((X_train,np.ones(X_train.shape[0]).reshape((X_train.shape[0],1))))
X_test = np.hstack((X_test,np.ones(X_test.shape[0]).reshape((X_test.shape[0],1))))
X_val = np.hstack((X_val,np.ones(X_val.shape[0]).reshape((X_val.shape[0],1))))

(11959, 28, 28)


In [ ]:
def sigmoidp(z):
  return 1/(1 + np.exp(-z))
def sigmoidn(z):
  return 1 - 1/(1 + np.exp(z))


In [ ]:
def sigmoid(z):
  z[np.where(z>=0)]=sigmoidp(z[np.where(z>=0)])
  z[np.where(z<0)]=sigmoidn(z[np.where(z<0)])
  return z


In [ ]:
def grad_sigmoid(z):
  return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def softmax(a):
  a=np.exp(a)
  return a/sum(a)

In [ ]:
def forward_prop(X,w,l):
  x=X
  layer_inputs=[]
  layer_inputs.append(x)
  for i in range(1,l-1): #l is no. of layers 
    x1=sigmoid(np.dot(w[i-1].T,x)) # add one to each x
    # for j in range(len(x1)):
    #   x1[j]=sigmoid(x1[j])
    #x1=transform(x1)
    layer_inputs.append(x1)
    x=x1
  layer_inputs.append(softmax(np.dot(w[l-2].T,x)))
  return layer_inputs
  



In [ ]:
def backward_prop(layer_inputs,w,Y):
  #last layer
  n=len(layer_inputs)-1
  w_new=w.copy()
  l=layer_inputs[-1]
  l1=layer_inputs[-2]
  W=w[-1]
  # k=(l-Y)*l*(1-l)
  k=l-Y
  # print(k.shape)
  delta=k
  dj_w=np.zeros_like(W)
  m,n1=W.shape
  # for i in range(0,m):
  #   for j in range(0,n1):
  #     # print(k[j])
  #     dj_w[i][j]=k[j]*l1[i]
  l1_n=np.reshape(l1,(m,1))
  k_n=np.reshape(k,(n1,1))
  dj_w=np.dot(l1_n,k_n.T)
  lambda1=0.001
  W=W-(lambda1*dj_w)
  w_new[-1]=W
  n=n-1
  # hidden layers
  while(n>0):
    # print(n)
    a=[]
    for j in range(len(layer_inputs[n])):
      z=0
      # for s in range(len(layer_inputs[n+1])):
      #   z+=delta[s]*w[n][j][s]
      z=np.sum(delta*w[n][j])
      a.append(z)
    a=np.array(a)
    l=layer_inputs[n]
    l1=layer_inputs[n-1]
    b=l*(1-l)
    delta=a*b
    k=delta
    W=w[n-1]
    dj_w=np.zeros_like(W)
    m,n1=W.shape
    # for i in range(0,m):
    #   for j in range(0,n1):
    #     dj_w[i][j]=k[j]*l1[i]
    l1_n=np.reshape(l1,(m,1))
    k_n=np.reshape(k,(n1,1))
    dj_w=np.dot(l1_n,k_n.T)
    lambda1=0.001
    W=W-(lambda1*dj_w)
    w_new[n-1]=W
    n=n-1
  return w_new





In [ ]:
def onehotencoding(a,max1):
  z=np.zeros(max1+1)
  z[a]=1
  return z

In [ ]:
w=[]
l=3
max_classes=int(max(Y_train))
nodes_list=[785,785,max_classes+1]
for i in range(len(nodes_list)-1):
  w.append(np.random.randn(nodes_list[i],nodes_list[i+1]))
epochs = 10

for i in range(epochs):
  for j in range(len(X_train)):
    Y = onehotencoding(Y_train[j],max_classes)
    layer_inputs = forward_prop(X_train[j],w,l)
    w = backward_prop(layer_inputs,w,Y)

In [ ]:
def accuracy(y_test,y_pred):
  count =0
  for i in range(len(y_pred)):
    if y_pred[i]==y_test[i]:
      count+=1
  accuracy = count*100/len(y_pred)
  return accuracy

In [ ]:
def predict(w,x,l):
  y=forward_prop(x,w,l)[-1]
  return np.argmax(y)

In [ ]:
y_pred=[]
for i in range(len(X_test)):
  y_pred.append(predict(w,X_test[i],l))
acc=accuracy(Y_test,y_pred)

In [ ]:
print(acc)

54.55607476635514


In [ ]:
y_pred=[]
for i in range(len(X_val)):
  y_pred.append(predict(w,X_val[i],l))
acc=accuracy(Y_val,y_pred)

In [ ]:
print(acc)

54.866997953814675


In [ ]:
y_pred=[]
for i in range(len(X_train)):
  y_pred.append(predict(w,X_train[i],l))
acc=accuracy(Y_train,y_pred)
print(acc)

80.75926080775984


In [ ]:
# np.savetxt("sample1.txt", w[1], delimiter =", ")